https://github.com/lmarza/CartPole-CNN/blob/main/dqnCartPole.py

In [1]:
import gym
import random
from keras import Sequential
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np

In [11]:
class DQN:

    """ Implementation of DQN algorithm """

    def __init__(self, action_space, state_space):

        self.action_space = action_space
        self.state_space = state_space
        self.epsilon = 1
        self.gamma = .95
        self.batch_size = 32
        self.epsilon_min = .01
        self.epsilon_decay = .995
        self.learning_rate = 0.001
        self.memory = deque(maxlen=10000)
        self.model = self.build_model()
        self.scores = []
        self.episodes = []
        self.average = []

    def build_model(self):

        model = Sequential()
        model.add(Dense(32, input_shape=(self.state_space,), activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):

        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self):

        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        states = np.squeeze(states)
        next_states = np.squeeze(next_states)

        targets = rewards + self.gamma*(np.amax(self.model.predict_on_batch(next_states), axis=1))*(1-dones)
        targets_full = self.model.predict_on_batch(states)

        ind = np.array([i for i in range(self.batch_size)])
        targets_full[[ind], [actions]] = targets

        self.model.fit(states, targets_full, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    


In [12]:
def train_dqn(episode):
    agent = DQN(env.action_space.n, env.observation_space.shape[0])
    scores = []
    for e in (range(episode)):
        state, info = env.reset()
        state = np.reshape(state, (1, 4))
        score = 0
        max_steps = 1000
        for i in range(max_steps):
            #env.render()
            action = agent.act(state)
            next_state, reward, done, truncated, info = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, 4))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            if done:
                scores.append(score)
                np.savetxt('result.out', scores, delimiter='\n')
                print("episode: {}/{}, score: {}".format(e, episode, score))
                #agent.PlotModel(score, e)
                break
    return scores  

In [13]:
env = gym.make('CartPole-v1')
np.random.seed(0)
ep = 300
scores = train_dqn(ep)

c:\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


episode: 0/300, score: 22.0
episode: 1/300, score: 17.0
episode: 2/300, score: 17.0
episode: 3/300, score: 25.0
episode: 4/300, score: 16.0
episode: 5/300, score: 10.0
episode: 6/300, score: 17.0
episode: 7/300, score: 13.0
episode: 8/300, score: 10.0
episode: 9/300, score: 11.0
episode: 10/300, score: 14.0
episode: 11/300, score: 12.0
episode: 12/300, score: 11.0
episode: 13/300, score: 29.0
episode: 14/300, score: 13.0
episode: 15/300, score: 13.0
episode: 16/300, score: 9.0
episode: 17/300, score: 10.0
episode: 18/300, score: 10.0
episode: 19/300, score: 11.0
episode: 20/300, score: 12.0
episode: 21/300, score: 13.0
episode: 22/300, score: 12.0
episode: 23/300, score: 10.0
episode: 24/300, score: 10.0
episode: 25/300, score: 11.0
episode: 26/300, score: 9.0
episode: 27/300, score: 9.0
episode: 28/300, score: 9.0
episode: 29/300, score: 12.0
episode: 30/300, score: 9.0
episode: 31/300, score: 18.0
episode: 32/300, score: 15.0
episode: 33/300, score: 12.0
episode: 34/300, score: 11.0


KeyboardInterrupt: 